In [1]:
import numba


In [2]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

@numba.njit
def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valores aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   return poblacionMatriz # No hacemos la conversión en lista de listas
                          # porque necesitamos de `numba`.


In [3]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y retorna una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''


@numba.njit
def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion):
    
    # Necesitamos el número de individuos de la población:
    numIndividuos = len(poblacion) 
    
    # Generamos un array con las evaluaciones en la función objetivo
    evaluaciones = np.empty(numIndividuos) # Primero generamos un array
                                           # vacío de tamaño adecuado.
    for i in range(numIndividuos):
        evaluaciones[i] = funcion(poblacion[i])

    # Generamos un array de las aptitudes de los individuos en la 
    # población.
    aptitudes = np.exp(1e-9 * (-evaluaciones))

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = np.sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = aptitudes / sumaAptitudes


    return probabilidades, masApto


In [4]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos retorna la que corresponde a la de las 
   probabilidades acumulativas.'''

@numba.njit
def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = np.cumsum(probabilidades)

    return probasAcumulativas # Retornamos las `probasAcumulativas` como
                              # un arreglo de NumPy (igual es iterable).


Padres mediante el uso de la ruleta:

In [5]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

@numba.njit
def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = np.random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   
   # Recorremos las probabilidades acumuladas para seleccionar el padre
   for i in range(len(probasAcumuladas)):
      if r <= probasAcumuladas[i] or np.isclose(r, probasAcumuladas[i], atol=tol):
         return i  # Índice del individuo seleccionado
               
   return -1  # En caso de que no se seleccione, aunque esto no debería suceder.


In [6]:
'''La segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k nuevos individuos, así que se van a 
   elegir k parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades acumulativas y un máximo de intentos para intentar evitar
   la generación de clones (en este caso, vamos a considerar un máximo
   de 100 para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''

@numba.njit
def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    
    numIndividuos = len(individuos)

    # Validación: si no hay suficientes individuos
    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return np.empty((0, 2, individuos.shape[1]))
        # Regresamos un array vacío

    # Como len(individuos) = 2k, le pedimos que elija len(individuos)/2 
    # parejas de padres.
    parejasNecesarias = numIndividuos // 2
    
    
    parejas_idxs = np.empty((parejasNecesarias, 2), dtype=np.int32)
    # Creamos un array para los índices de tamaño adecuado.
    # Como queremos parejas, el array tiene dos columnas, la idea es que
    # cada fila del array represente una pareja de padres.

    '''En otras palabras, `parejas_idxs` tendrá la forma:
       [[padre1_idx1, padre2_idx1],
       [padre1_idx2, padre2_idx2],
       [padre1_idx3, padre2_idx3],
       ...
       [padre1_idxN, padre2_idxN]]'''


    for i in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)
            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salimos del ciclo
            intentos += 1 # No encontramos un padre distinto, hacemos 
                          # otro intento.
            if intentos >= maxIntentos: # Rebasamos los intentos que
                                        # teníamos permitidos.
                padre2_idx = padre1_idx # Aceptamos el clon.
                break

        parejas_idxs[i, 0] = padre1_idx # Primer padre se asigna en la
                                        # fila `i` de la primer columna.
        parejas_idxs[i, 1] = padre2_idx # Segundo padre se asigna en la
                                        # fila `i` de la segunda columna.

    # Ahora construimos el array de parejas usando los índices generados
    # para almacenar las parejas de individuos.
    parejas = np.empty((parejasNecesarias, 2, individuos.shape[1]), dtype=individuos.dtype)
    
    for i in range(parejasNecesarias):
        parejas[i, 0] = individuos[parejas_idxs[i, 0]]  # Primer padre
        parejas[i, 1] = individuos[parejas_idxs[i, 1]]  # Segundo padre

    
    return parejas


**Ojo:** Vas a tener que volver a definir las funciones de prueba, pero sólo hazlo si ves que de veras mejoraste en los tiempos. Si no, no hagas nada.

In [7]:
@numba.njit
def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando n_bit bits
    y una partición uniforme en [a, b].

    Parámetros:
    x: número real a codificar.
    n_bit: número de bits a utilizar.
    intervalo: (a, b), extremos del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con n_bit bits
    en el intervalo [a, b].
    """
    a, b = intervalo

    # Calcula la precisión de la representación.
    precision = (b - a) / ((2 ** n_bit) - 1)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = np.zeros(n_bit, dtype=np.int32)
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    return x_binario

@numba.njit
def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios
    utilizando n_bit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    n_bit: Número de bits a utilizar para las entradas de nuestro arreglo.
    intervalo: (a, b), extremos del intervalo.

    Retorno:
    Arreglo de arreglos binarios, donde cada sub-arreglo representa un número real
    codificado en binario.
    """
    dim_x = vector_reales.shape[0]

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = np.empty((dim_x, n_bit), dtype=np.int32)

    for i in range(dim_x):
        vector_binario[i] = codifica_real(vector_reales[i], n_bit, intervalo)

    return vector_binario


In [8]:
@numba.njit
def Padres_Binarios(parejas, nBits, intervalo):
    """
    Codifica parejas de vectores en binario.
    """
    numParejas = parejas.shape[0]
    parejasBinarias = np.empty((numParejas, 2, parejas.shape[2], nBits), dtype=np.int32)

    for i in range(numParejas):
        padre1 = parejas[i, 0]  # Vector 1
        padre2 = parejas[i, 1]  # Vector 2

        # Codifica cada padre en binario y lo almacena.
        parejasBinarias[i, 0] = codifica_vector(padre1, nBits, intervalo)
        parejasBinarias[i, 1] = codifica_vector(padre2, nBits, intervalo)

    return parejasBinarias



In [9]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")

'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 70
n = 10
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 17

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")


Semilla generada: 1728766569
Hora de generación: 2024-10-12 14:56:09

La población inicial fue:
[[ 1.08407976  1.46567255 -1.56660131 ... -0.84489205 -0.19224332
  -0.92748761]
 [ 0.34512669  1.26264477  0.12219989 ...  0.8733752   0.92354572
  -1.49700272]
 [-0.81721431 -0.76939921 -0.62778986 ... -0.65243929 -1.8358912
   0.26707455]
 ...
 [-1.09891472 -1.88050521 -1.86593961 ...  0.12568228  0.82607718
   1.54511542]
 [-1.12023769 -1.92408183 -0.90426796 ... -0.64920665  1.9662456
  -0.5552101 ]
 [ 1.84749532  1.44786223  0.88511591 ...  0.64383268  0.78733131
  -1.35780329]]

Sus probabilidades de selección fueron:
[0.00714287 0.00714288 0.00714287 0.00714287 0.00714285 0.00714287
 0.00714284 0.00714282 0.00714286 0.00714286 0.00714286 0.00714283
 0.00714283 0.00714288 0.00714286 0.00714287 0.00714287 0.00714284
 0.00714283 0.00714286 0.00714286 0.00714288 0.00714287 0.00714287
 0.00714285 0.00714286 0.00714286 0.00714282 0.00714285 0.00714286
 0.00714288 0.00714286 0.00714286 0.00

**Prueba de fuego, porque este antes no jalaba:**

In [10]:
import numpy as np
import numba

@numba.njit
def Cruza_N_Puntos_np_numba(padres, nPuntos):
    numParejas = padres.shape[0]  # Número de parejas de padres
    numCromosomas = padres[0, 0].shape[0]  # Número de cromosomas por padre (filas)
    numGenes = padres[0, 0].shape[1]  # Número de genes por cromosoma (columnas)

    # Inicializamos un array vacío para almacenar los hijos
    hijos = np.empty((2 * numParejas, numCromosomas, numGenes), dtype=np.int32)

    # Iteramos sobre cada pareja de padres
    for pareja_idx in range(numParejas):
        padre1 = padres[pareja_idx, 0]
        padre2 = padres[pareja_idx, 1]

        # Generamos los arrays vacíos para los hijos de la pareja actual
        hijo1 = np.empty_like(padre1)
        hijo2 = np.empty_like(padre2)

        # Generar n puntos de corte únicos
        puntosCorte = np.random.choice(np.arange(1, numGenes), nPuntos, replace=False)
                      # `replace=False` es para garantizar que los puntos de corte sí
                      # sean diferentes.
        puntosCorte.sort()  # Asegura que los puntos de corte están ordenados.

        # Iteramos sobre cada cromosoma de los padres
        for cromo_idx in range(numCromosomas):
            cromo1 = padre1[cromo_idx]
            cromo2 = padre2[cromo_idx]

            # Alternamos segmentos entre los padres
            ultimo_punto = 0
            switch = False

            for punto in np.append(puntosCorte, numGenes):  # Añadimos el último punto
                if switch:
                    hijo1[cromo_idx, ultimo_punto:punto] = cromo2[ultimo_punto:punto]
                    hijo2[cromo_idx, ultimo_punto:punto] = cromo1[ultimo_punto:punto]
                else:
                    hijo1[cromo_idx, ultimo_punto:punto] = cromo1[ultimo_punto:punto]
                    hijo2[cromo_idx, ultimo_punto:punto] = cromo2[ultimo_punto:punto]

                switch = not switch
                ultimo_punto = punto

        # Guardamos los hijos generados en el array de hijos
        hijos[2 * pareja_idx] = hijo1
        hijos[2 * pareja_idx + 1] = hijo2

    return hijos


In [14]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")

'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 2 
n = 3
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 5
nPuntos = 3

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial de {2*k} individuos fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")

hijosBinarios = Cruza_N_Puntos_np_numba(parejasBinarias, nPuntos)
print(f"Los hijos en binario fueron:\n{hijosBinarios}\n")
print(f"Tienes {len(hijosBinarios)} hijos, antes de la eliminación de uno al azar.\n")


Semilla generada: 1728766597
Hora de generación: 2024-10-12 14:56:37

La población inicial de 4 individuos fue:
[[-1.61027888 -0.90440485 -0.51229674]
 [ 2.02864108  0.85548321  1.29379048]
 [ 0.20116479 -0.24312838  1.17111663]
 [ 1.2283542  -0.00404     0.18021154]]

Sus probabilidades de selección fueron:
[0.24999983 0.24999991 0.25000015 0.25000012]

El más apto fue:
[ 0.20116479 -0.24312838  1.17111663]

Las probabilidades acumulativas fueron:
[0.24999983 0.49999973 0.74999988 1.        ]

Las parejas seleccionadas para la reproducción fueron:
[[[ 2.02864108  0.85548321  1.29379048]
  [-1.61027888 -0.90440485 -0.51229674]]

 [[ 0.20116479 -0.24312838  1.17111663]
  [-1.61027888 -0.90440485 -0.51229674]]]

Los padres en binario fueron:
[[[[1 1 1 1 0]
   [1 0 1 0 1]
   [1 1 0 0 1]]

  [[0 0 0 1 1]
   [0 1 0 0 0]
   [0 1 0 1 1]]]


 [[[1 0 0 0 1]
   [0 1 1 0 1]
   [1 1 0 0 0]]

  [[0 0 0 1 1]
   [0 1 0 0 0]
   [0 1 0 1 1]]]]

Los hijos en binario fueron:
[[[1 0 0 1 1]
  [1 1 0 0 0]
 

**Sí funciona, hasta aquí todo funciona.**

In [15]:
import numpy as np

@numba.njit
def Mutador_1_flip_np_numba(hijosBinarios, probaMutacion):
    
    # Obtenemos el número de individuos y cromosomas
    numHijos = hijosBinarios.shape[0]
    numCromosomas = hijosBinarios.shape[1]
    
    numBits = hijosBinarios.shape[2]
    
    # Recorremos cada individuo
    for i in range(numHijos):
        # Recorremos cada cromosoma del individuo
        for j in range(numCromosomas):
            # Generamos un número aleatorio
            if np.random.rand() < probaMutacion:
                # Elegimos un bit al azar dentro del cromosoma
                bit_a_flip = np.random.randint(0, numBits)
                # Mutar el bit
                hijosBinarios[i, j, bit_a_flip] ^= 1  # Flip bit
                # Modificamos directamente sobre hijosBinarios porque
                # a los que realmente consideraremos como hijos en cada
                # generación es a los generados después de la reproducción
                # y mutación.

    return hijosBinarios

# Ejemplo de uso
individuals = np.array([
    [[0, 0, 0, 0, 1],
     [1, 1, 1, 0, 0],
     [1, 0, 1, 0, 1]],

    [[0, 0, 1, 0, 1],
     [1, 0, 0, 0, 1],
     [0, 1, 1, 1, 0]],

    [[0, 0, 1, 0, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 0, 0]],

    [[0, 0, 0, 0, 1],
     [0, 1, 0, 0, 1],
     [1, 0, 0, 0, 0]]
])

# Definir la probabilidad de mutación
mutation_probability = 0.2  # 20% de probabilidad de mutación

# Mutar los individuos
mutated_individuals = Mutador_1_flip_np_numba(individuals, mutation_probability)

# Imprimir resultados
print("Individuos originales:\n", individuals)
print("Individuos mutados:\n", mutated_individuals)


Individuos originales:
 [[[0 0 0 0 1]
  [0 1 1 0 0]
  [1 0 1 0 1]]

 [[0 0 1 0 1]
  [1 0 0 0 1]
  [0 1 1 1 0]]

 [[0 0 1 0 0]
  [1 0 0 0 1]
  [1 1 1 0 0]]

 [[0 0 0 0 1]
  [0 1 0 0 1]
  [1 0 0 0 0]]]
Individuos mutados:
 [[[0 0 0 0 1]
  [0 1 1 0 0]
  [1 0 1 0 1]]

 [[0 0 1 0 1]
  [1 0 0 0 1]
  [0 1 1 1 0]]

 [[0 0 1 0 0]
  [1 0 0 0 1]
  [1 1 1 0 0]]

 [[0 0 0 0 1]
  [0 1 0 0 1]
  [1 0 0 0 0]]]


In [18]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")

'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 35 
n = 10
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 20
nPuntos = 16
probaMutacion = 0.1

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial de {2*k} individuos fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")

hijosBinarios = Cruza_N_Puntos_np_numba(parejasBinarias, nPuntos)
#print(f"Los hijos en binario fueron:\n{hijosBinarios}\n")
print(f"Tienes {len(hijosBinarios)} hijos, antes de la eliminación de uno al azar.\n")

hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)
print(f"Los hijos en binario (ya mutados) fueron:\n{hijosMutados}\n")


Semilla generada: 1728766632
Hora de generación: 2024-10-12 14:57:12

La población inicial de 70 individuos fue:
[[-2.04285987e+00  1.79584726e+00  1.68878188e+00  1.76168346e+00
  -5.67580210e-01 -1.96984354e+00 -9.11171328e-01 -1.58752999e+00
  -1.03126387e-01 -5.97629157e-01]
 [-1.16470003e+00 -1.81236741e+00 -2.99085611e-01 -6.79278798e-01
   1.54936913e+00 -1.72754717e+00  3.76789293e-02 -1.28871823e+00
  -2.94805034e-01 -8.85960492e-01]
 [-1.59456437e+00 -7.24726927e-01  1.85774056e+00 -2.57543057e-01
   3.38845687e-01  1.31132642e+00 -5.92412154e-01  3.33529876e-01
   1.22200528e+00 -1.20010293e+00]
 [-9.87039176e-01 -6.17157390e-01 -1.55056558e+00  1.48318517e+00
  -1.16346095e+00  6.76003358e-01 -1.48607942e+00  6.57034577e-01
  -2.76681475e-01  1.22476128e+00]
 [ 1.83397531e+00  6.82469536e-01  1.12053182e+00 -2.56218824e-01
  -1.15591840e+00  7.37034477e-01  1.92461006e+00 -1.19755630e+00
  -1.00387153e+00  1.47770189e-01]
 [-1.08272035e+00  1.21622260e+00 -8.58281829e-01  2

**HASTA AQUÍ FUNCIONA BIEN**

In [23]:
@numba.njit
def Hijos_Decodificados(hijosMutados, nBits, intervalo):
    """
    Decodifica un array de hijos mutados en números reales.

    Parámetros:
    hijosMutados: np.array de hijos mutados, donde cada hijo es un vector de bits
    nBits: número de bits a utilizar para cada número
    intervalo: de la forma [a, b]

    Retorno:
    np.array de números reales decodificados
    """
    
    num_hijos = hijosMutados.shape[0]      # Obtenemos el número de hijos
    num_cromosomas = hijosMutados.shape[1]  # Obtenemos el número de cromosomas
    
    # Crea un array vacío para los hijos decodificados
    hijosDecodificados = np.empty((num_hijos, num_cromosomas))  # De la forma correcta

    for i in range(num_hijos):
        for j in range(num_cromosomas):
            hijoMutado = hijosMutados[i, j]  # Acceder al cromosoma
            
            # Decodificación directa del cromosoma
            a, b = intervalo
            precision = (b - a) / ((2 ** nBits) - 1)  # Calcula la precisión de la representación
            # Convierte el vector de bits en un número entero
            indice = 0
            for bit in range(nBits):
                indice += hijoMutado[bit] * (2 ** (nBits - 1 - bit))  # Cálculo del valor decimal
            hijoDecodificado = a + indice * precision  # Calcula el número real decodificado
            
            # Almacena el resultado en el array de salida
            hijosDecodificados[i, j] = hijoDecodificado

    return hijosDecodificados


In [24]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")

'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 2 
n = 3
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 5
nPuntos = 3
probaMutacion = 0.1

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial de {2*k} individuos fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")

hijosBinarios = Cruza_N_Puntos_np_numba(parejasBinarias, nPuntos)
#print(f"Los hijos en binario fueron:\n{hijosBinarios}\n")
print(f"Tienes {len(hijosBinarios)} hijos, antes de la eliminación de uno al azar.\n")

hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)
print(f"Los hijos en binario (ya mutados) fueron:\n{hijosMutados}\n")

hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
print(f"Los hijos decodificados son:\n{hijosDecodificados}")


Semilla generada: 1728768953
Hora de generación: 2024-10-12 15:35:53

La población inicial de 4 individuos fue:
[[ 0.19026415  1.82468181  1.69364354]
 [ 1.72208124 -0.64182341  0.5013051 ]
 [-1.475026   -1.78214348 -1.84204169]
 [-1.04467794 -1.13017209  0.45197067]]

Sus probabilidades de selección fueron:
[0.25000026 0.25000008 0.24999939 0.25000027]

El más apto fue:
[-1.04467794 -1.13017209  0.45197067]

Las probabilidades acumulativas fueron:
[0.25000026 0.50000035 0.74999973 1.        ]

Las parejas seleccionadas para la reproducción fueron:
[[[-1.04467794 -1.13017209  0.45197067]
  [ 0.19026415  1.82468181  1.69364354]]

 [[-1.04467794 -1.13017209  0.45197067]
  [ 0.19026415  1.82468181  1.69364354]]]

Los padres en binario fueron:
[[[[0 0 1 1 1]
   [0 0 1 1 0]
   [1 0 0 1 0]]

  [[1 0 0 0 0]
   [1 1 1 0 1]
   [1 1 1 0 0]]]


 [[[0 0 1 1 1]
   [0 0 1 1 0]
   [1 0 0 1 0]]

  [[1 0 0 0 0]
   [1 1 1 0 1]
   [1 1 1 0 0]]]]

Tienes 4 hijos, antes de la eliminación de uno al azar.

L

Funciones de reemplazo generacional:

In [25]:
'''Función que realiza el reemplazo con elitismo.'''

@numba.njit
def Reemplazo_Elitismo(hijosDecodificados, masApto):

    numHijos, dimension = hijosDecodificados.shape

    # Elegimos el índice de un hijo al azar para eliminarlo:
    idx_aleatorio = np.random.randint(0, numHijos)

    # Creamos un nuevo array sin el hijo borrado:
    hijosReducidos = np.empty((numHijos-1, dimension))
    contador = 0
    for i in range(numHijos):
        if i != idx_aleatorio:
            hijosReducidos[contador, :] = hijosDecodificados[i, :]
            contador += 1

    # Creamos un nuevo array con el más apto antes de la reproducción:
    siguienteGeneracion = np.empty((numHijos, dimension))
    for i in range(numHijos-1):
        siguienteGeneracion[i, :] = hijosReducidos[i, :]
    siguienteGeneracion[numHijos-1, :] = masApto

    return siguienteGeneracion


In [37]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")

'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 35 
n = 10
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 20
nPuntos = 14
probaMutacion = 0.1

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial de {2*k} individuos fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")

hijosBinarios = Cruza_N_Puntos_np_numba(parejasBinarias, nPuntos)
#print(f"Los hijos en binario fueron:\n{hijosBinarios}\n")
print(f"Tienes {len(hijosBinarios)} hijos, antes de la eliminación de uno al azar.\n")

hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)
print(f"Los hijos en binario (ya mutados) fueron:\n{hijosMutados}\n")

hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
print(f"Los hijos decodificados son:\n{hijosDecodificados}\n")

siguienteGeneracionElitista = Reemplazo_Elitismo(hijosDecodificados, masApto)
print(f"La siguiente generación es:\n{siguienteGeneracionElitista}\n")


Semilla generada: 1728769947
Hora de generación: 2024-10-12 15:52:27

La población inicial de 70 individuos fue:
[[ 1.58338898 -1.09696792  1.30129451  1.79081548  0.62179496  0.99578037
  -1.13575351 -1.33287888 -0.13186468 -1.63187867]
 [ 0.41066492  0.08206561 -0.38155443 -2.03627751  0.50780536 -1.35234846
  -0.64453437  1.0555373   0.86128719 -1.25066567]
 [ 1.79908238  1.7382953  -0.70624769 -0.65557371  0.225569   -1.0940531
  -1.6757243  -0.38872447 -0.19080636 -0.91007033]
 [-0.29268203 -1.32172975  1.73541525  0.0615718   1.22890217 -1.19126804
   0.42814964 -0.57195635 -0.30369147 -1.13009255]
 [ 1.36496248  0.18169707 -1.2931541  -1.71926153  1.82009124  1.32346441
  -0.69828068  0.49320207  2.03687096  0.1240911 ]
 [ 1.78343274 -0.51346793  0.12750637  1.8967454   2.00122484 -0.5234954
  -0.87806092 -2.02390063 -0.43571128  1.44281769]
 [ 0.65888939  1.3819968  -1.70498418  0.48066287 -0.36928596 -0.57200666
  -0.39315091  0.60287318  0.41180313  1.88779226]
 [-1.71224783 

In [78]:
@numba.njit
def AG_Reemplazo_Elitismo(funcion, intervalo, dimension, 
                          kindividuos, nBits, nCortes, 
                          probaMutacion, iteraciones):

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo)

    # Repetiremos esto mientras tengamos iteraciones:
    for i in range(iteraciones):

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres:
        padres = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruza_N_Puntos_np_numba(padresBinarios, nCortes)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
        
        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Reemplazo_Elitismo(hijosDecodificados, masApto)

    return masApto


In [80]:
kindividuos = 35 
dimension = 10
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 20
nCortes = 15
probaMutacion = 0.02
iteraciones = 1000

masApto = AG_Reemplazo_Elitismo(funcion, intervalo, dimension, 
                          kindividuos, nBits, nCortes, 
                          probaMutacion, iteraciones)

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")

print(f"Para la función {funcion}, el más apto después\nde {iteraciones} iteraciones fue:\n{masApto}\nCon una evaluación de {funcion(masApto)}")


Semilla generada: 1728771249
Hora de generación: 2024-10-12 16:14:09

Para la función CPUDispatcher(<function Rosenbrock at 0x000001A9BBA165C0>), el más apto después
de 1000 iteraciones fue:
[ 5.11893066e-01  2.56595948e-01 -4.43359798e-04  8.75196147e-03
 -2.09961138e-03  1.06504008e-02  6.30664664e-03  6.12305271e-03
  6.22070906e-03 -1.67441566e-02]
Con una evaluación de 8.221507260563772
